<a href="https://colab.research.google.com/github/wlswjdals/project_braille01/blob/main/braille.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install firebase-admin
!pip install watchdog pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 49.3 MB/s eta 0:00:00


In [ ]:
## 1번


# 구글 드라이브에 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## 2번

import firebase_admin
from firebase_admin import credentials, db
import requests

# 서비스 계정 키 파일 경로
cred = credentials.Certificate("/braille-database-firebase-adminsdk-fcowu-411df8ca35.json")

# Firebase Admin SDK 초기화
firebase_admin.initialize_app(cred, {
    'storageBucket': 'gs://braille-database.appspot.com',  # Cloud Storage 버킷 이름
    'databaseURL': 'https://braille-database-default-rtdb.firebaseio.com/'               # Realtime Database URL
})


In [ ]:
## 실행 X

# Firebase Realtime Database에서 다운로드 URL 가져오기
def get_download_urls():
    ref = db.reference('files')  # 'files' 노드를 참조
    files = ref.get()  # 모든 파일 데이터 가져오기

    if files:
        for key, file_data in files.items():
            file_name = file_data.get('fileName')
            file_url = file_data.get('fileUrl')
            print(f"File Name: {file_name}")
            print(f"Download URL: {file_url}")
            print('---')
            # 파일 다운로드
            download_file(file_url, file_name)
    else:
        print("No files found in the database.")

# 파일 다운로드
def download_file(file_url, destination_path):
    response = requests.get(file_url)

    if response.status_code == 200:
        with open(destination_path, 'wb') as file:
            file.write(response.content)
        print(f"File downloaded to {destination_path}")
    else:
        print("Failed to download file.")

# Firebase에서 다운로드 URL 가져오기 및 파일 다운로드
get_download_urls()

File Name: uploaded_file.pdf
Download URL: https://firebasestorage.googleapis.com/v0/b/braille-database.firebasestorage.app/o/uploads%2Fuploaded_file.pdf?alt=media&token=ba59e464-91a3-433c-a9cb-d18d8a6fbdc7
---
File downloaded to uploaded_file.pdf


In [ ]:
## 실행 X


# Firebase에서 가장 최근 파일 가져오기 및 다운로드
def get_latest_file():
    ref = db.reference('files')  # 'files' 노드를 참조
    files = ref.order_by_key().get()  # 키 기준으로 정렬된 파일 데이터 가져오기

    if files:
        # 가장 최근 파일 데이터 (마지막 키)
        last_key = list(files.keys())[-1]
        latest_file_data = files[last_key]

        file_name = latest_file_data.get('fileName')
        file_url = latest_file_data.get('fileUrl')

        print(f"Latest File Name: {file_name}")
        print(f"Download URL: {file_url}")

        # 파일 다운로드
        download_file(file_url, file_name)
    else:
        print("No files found in the database.")

# 파일 다운로드 함수
def download_file(file_url, destination_path):
    response = requests.get(file_url)

    if response.status_code == 200:
        with open(destination_path, 'wb') as file:
            file.write(response.content)
        print(f"File downloaded to {destination_path}")
    else:
        print("Failed to download file.")

# Firebase에서 가장 최근 파일 가져오기 및 다운로드 실행
get_latest_file()


Latest File Name: FixedFileName.jpg
Download URL: https://firebasestorage.googleapis.com/v0/b/braille-database.firebasestorage.app/o/uploads%2FFixedFileName.jpg?alt=media&token=60319b9e-d66b-48a7-96b5-c215f64d26d4
File downloaded to FixedFileName.jpg


In [ ]:
## 3번  최종  ##

import os
import requests
from google.colab import files
# /content/ 디렉토리 비우기
def clear_content_directory():
    content_dir = '/content/'
    for filename in os.listdir(content_dir):
        file_path = os.path.join(content_dir, filename)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)  # 파일 삭제
            elif os.path.isdir(file_path):
                os.rmdir(file_path)  # 빈 디렉토리 삭제
        except Exception as e:
            print(f"Error removing file {file_path}: {e}")

# Firebase에서 다운로드 URL 가져오기 및 파일 다운로드
def get_download_urls():
    ref = db.reference('files')  # 'files' 노드를 참조
    files = ref.get()  # 모든 파일 데이터 가져오기

    if files:
        # 가장 마지막 파일의 다운로드 URL을 가져오기 위해 마지막 파일만 처리
        last_file = list(files.items())[-1]  # 마지막 파일
        file_data = last_file[1]
        file_name = file_data.get('fileName')
        file_url = file_data.get('fileUrl')

        print(f"File Name: {file_name}")
        print(f"Download URL: {file_url}")

        # 파일 다운로드
        download_file(file_url, f'/content/{file_name}')
    else:
        print("No files found in the database.")

# 파일 다운로드
def download_file(file_url, destination_path):
    response = requests.get(file_url)

    if response.status_code == 200:
        with open(destination_path, 'wb') as file:
            file.write(response.content)
        print(f"File downloaded to {destination_path}")
    else:
        print("Failed to download file.")

# /content/ 디렉토리 비우기
clear_content_directory()

# Firebase에서 다운로드 URL 가져오기 및 파일 다운로드
get_download_urls()

Error removing file /content/.config: [Errno 39] Directory not empty: '/content/.config'
Error removing file /content/drive: [Errno 16] Device or resource busy: '/content/drive'
Error removing file /content/sample_data: [Errno 39] Directory not empty: '/content/sample_data'
No files found in the database.


In [ ]:
## colab에서 firebase로 문자 보내기

ref = db.reference('message')
ref.set("2")

print("Message sent to Firebase!")

Message sent to Firebase!


In [ ]:
!pip install firebase-admin
!pip install watchdog pdfplumber

import os
import time
import requests
import pdfplumber
import hashlib  # 추가
import firebase_admin
from firebase_admin import credentials, db
from google.colab import files

# 구글 드라이브에 마운트
from google.colab import drive
drive.mount('/content/drive')

# Firebase 초기화
cred = credentials.Certificate("/braille-database-firebase-adminsdk-fcowu-411df8ca35.json")
firebase_admin.initialize_app(cred, {
    'storageBucket': 'gs://braille-database.appspot.com',
    'databaseURL': 'https://braille-database-default-rtdb.firebaseio.com/'
})

# 마지막으로 처리한 파일 해시값 저장 (중복 방지)
last_processed_hash = None

# PDF에서 텍스트 추출
def extract_text_from_pdf(file_path):
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                text += page.extract_text()
        return text
    except Exception as e:
        print(f"OCR 오류: {e}")
        return None

# 파일 해시값 계산
def calculate_file_hash(file_path):
    """파일 내용의 SHA256 해시값 계산"""
    hash_sha256 = hashlib.sha256()
    try:
        with open(file_path, "rb") as f:
            for chunk in iter(lambda: f.read(4096), b""):
                hash_sha256.update(chunk)
        return hash_sha256.hexdigest()
    except Exception as e:
        print(f"해시 계산 오류: {e}")
        return None

# Firebase에서 다운로드 URL 가져오기 및 파일 다운로드
def get_download_urls():
    ref = db.reference('files')
    files = ref.get()

    if files:
        # 가장 최신 파일 데이터 가져오기
        last_file = list(files.items())[-1]
        file_data = last_file[1]
        file_name = file_data.get('fileName')
        file_url = file_data.get('fileUrl')
        return file_name, file_url
    else:
        print("Firebase 데이터베이스에 파일이 없습니다.")
        return None, None

# 파일 다운로드
def download_file(file_url, destination_path):
    try:
        response = requests.get(file_url)
        if response.status_code == 200:
            with open(destination_path, 'wb') as file:
                file.write(response.content)
            print(f"파일 다운로드 완료: {destination_path}")
            return True
        else:
            print(f"파일 다운로드 실패: HTTP {response.status_code}")
            return False
    except Exception as e:
        print(f"파일 다운로드 중 오류 발생: {e}")
        return False

# PDF 파일 OCR 처리
def process_pdf(file_name, file_path):
    print(f"OCR 처리 시작: {file_name}")
    text = extract_text_from_pdf(file_path)
    if text:
        # 5글자 제한
        limited_text = text[:30]
        print(f"OCR 결과 (30글자 제한): {limited_text}")
        # Firebase에 결과 저장
        ref = db.reference('message')
        ref.set(limited_text)
        print("Firebase에 OCR 결과 저장 완료.")
    else:
        print(f"OCR 실패: {file_name}")

# Firebase에서 최신 파일 가져오기 및 OCR 실행
def monitor_and_process():
    global last_processed_hash
    print("Firebase 파일 감시 시작...")
    while True:
        file_name, file_url = get_download_urls()
        if file_name and file_url:
            # 파일 경로 설정
            destination_path = f"/content/{file_name}"
            if download_file(file_url, destination_path):  # 파일 다운로드 성공 시
                # 파일 해시값 계산
                current_hash = calculate_file_hash(destination_path)
                if current_hash != last_processed_hash:
                    print(f"새 파일 또는 파일 내용 변경 감지: {file_name}")
                    process_pdf(file_name, destination_path)  # OCR 처리
                    last_processed_hash = current_hash  # 처리 완료 파일의 해시값 갱신
                else:
                    print("파일 내용이 이전과 동일합니다. OCR을 건너뜁니다.")
        else:
            print("새로운 파일이 없습니다.")
        time.sleep(10)  # 10초 간격으로 Firebase 확인

# OCR 감시 및 실행 시작
monitor_and_process()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Firebase 파일 감시 시작...
파일 다운로드 완료: /content/uploaded_file.jpg
새 파일 또는 파일 내용 변경 감지: uploaded_file.jpg
OCR 처리 시작: uploaded_file.jpg
OCR 오류: No /Root object! - Is this really a PDF?
OCR 실패: uploaded_file.jpg
파일 다운로드 완료: /content/uploaded_file.jpg
파일 내용이 이전과 동일합니다. OCR을 건너뜁니다.
파일 다운로드 완료: /content/uploaded_file.jpg
파일 내용이 이전과 동일합니다. OCR을 건너뜁니다.
파일 다운로드 완료: /content/uploaded_file.jpg
파일 내용이 이전과 동일합니다. OCR을 건너뜁니다.
파일 다운로드 완료: /content/uploaded_file.jpg
파일 내용이 이전과 동일합니다. OCR을 건너뜁니다.
파일 다운로드 완료: /content/uploaded_file.jpg
파일 내용이 이전과 동일합니다. OCR을 건너뜁니다.
파일 다운로드 완료: /content/uploaded_file.jpg
파일 내용이 이전과 동일합니다. OCR을 건너뜁니다.
파일 다운로드 완료: /content/uploaded_file.jpg
파일 내용이 이전과 동일합니다. OCR을 건너뜁니다.
파일 다운로드 완료: /content/uploaded_file.jpg
파일 내용이 이전과 동일합니다. OCR을 건너뜁니다.
파일 다운로드 완료: /content/uploaded_file.jpg
파일 내용이 이전과 동일합니다. OCR을 건너뜁니다.
파일 다운로드 완료: /content/uploaded_file.jpg
파일 내용이 이